### Simple RAG Demo

In [18]:
import dotenv

from autogen_core.models import UserMessage, AssistantMessage
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

dotenv.load_dotenv()

token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4o-mini",
    model="gpt-4o-mini",
    api_version=dotenv.get_key(dotenv.find_dotenv(), "AZURE_AUTH_API_VERSION"),
    azure_endpoint=dotenv.get_key(dotenv.find_dotenv(), "AZURE_AUTH_API_BASE"),
    api_key=dotenv.get_key(dotenv.find_dotenv(), "AZURE_AUTH_API_KEY")
)

In [10]:

import pandas as pd

data = {
    "Name": ["Joe's Pizza", "Lombardi's", "Di Fara Pizza", "Prince Street Pizza", "Lucali", "John's of Bleecker Street", "Patsy's Pizzeria", "Roberta's", "Artichoke Basille's Pizza", "Grimaldi's"],
    "Address": ["7 Carmine St, New York, NY", "32 Spring St, New York, NY", "1424 Avenue J, Brooklyn, NY", "27 Prince St A, New York, NY", "575 Henry St, Brooklyn, NY", "278 Bleecker St, New York, NY", "2287 1st Avenue, New York, NY", "261 Moore St, Brooklyn, NY", "321 E 14th St, New York, NY", "1 Front St, Brooklyn, NY"],
    "Favourite Ingredient": ["Pepperoni", "Cheese", "Mushroom", "Pepperoni", "Mushroom", "Pepperoni", "Cheese", "Mushroom", "Pepperoni", "Mushroom"],
    "City": ["New York", "New York", "Brooklyn", "New York", "Brooklyn", "New York", "New York", "Brooklyn", "New York", "Brooklyn"],
    "State": ["NY", "NY", "NY", "NY", "NY", "NY", "NY", "NY", "NY", "NY"],
    "Zip Code": ["10014", "10012", "11230", "10012", "11231", "10014", "10035", "11206", "10003", "11201"],
    "Phone Number": ["(212) 366-1182", "(212) 941-7994", "(718) 258-1367", "(212) 966-4100", "(718) 858-4086", "(212) 243-1680", "(212) 534-9783", "(718) 417-1118", "(212) 228-2004", "(718) 858-4300"],
    "Rating": [4.5, 4.3, 4.6, 4.4, 4.7, 4.4, 4.2, 4.5, 4.3, 4.4],
    "Price Range": ["$", "$$", "$$$", "$$", "$$$", "$$", "$", "$$", "$$", "$"],
    "Delivery": [True, True, False, True, False, False, True, True, True, True],
    "Website": ["joespizzanyc.com", "firstpizza.com", "difarapizzany.com", "princestreetpizzanyc.com", "lucali.com", "johnsbrickovenpizza.com", "thepatsyspizza.com", "robertaspizza.com", "artichokepizza.com", "grimaldis-pizza.com"]
}

df = pd.DataFrame(data)

df.head()

,Name,Address,Favourite Ingredient,City,State,Zip Code,Phone Number,Rating,Price Range,Delivery,Website
0,Joe's Pizza,"7 Carmine St, New York, NY",Pepperoni,New York,NY,10014,(212) 366-1182,4.5,$,True,joespizzanyc.com
1,Lombardi's,"32 Spring St, New York, NY",Cheese,New York,NY,10012,(212) 941-7994,4.3,$$,True,firstpizza.com
2,Di Fara Pizza,"1424 Avenue J, Brooklyn, NY",Mushroom,Brooklyn,NY,11230,(718) 258-1367,4.6,$$$,False,difarapizzany.com
3,Prince Street Pizza,"27 Prince St A, New York, NY",Pepperoni,New York,NY,10012,(212) 966-4100,4.4,$$,True,princestreetpizzanyc.com
4,Lucali,"575 Henry St, Brooklyn, NY",Mushroom,Brooklyn,NY,11231,(718) 858-4086,4.7,$$$,False,lucali.com


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(
    df['Name'] + " " + 
    df['Favourite Ingredient'] + " " + 
    df['Address'] + " " + 
    df['City'] + " " + 
    df['Website'] + " "
)

In [17]:
def find_similar_pizzas(query):
    query_vec = vectorizer.transform([query])
    similarity = cosine_similarity(query_vec, tfidf_matrix).flatten()
    indices = similarity.argsort()[-3:][::-1]
    return df.iloc[indices]

query = "get pizza places in brooklyn that like mushrooms"
shroomers = find_similar_pizzas(query)

shroomers.head()

,Name,Address,Favourite Ingredient,City,State,Zip Code,Phone Number,Rating,Price Range,Delivery,Website
9,Grimaldi's,"1 Front St, Brooklyn, NY",Mushroom,Brooklyn,NY,11201,(718) 858-4300,4.4,$,True,grimaldis-pizza.com
2,Di Fara Pizza,"1424 Avenue J, Brooklyn, NY",Mushroom,Brooklyn,NY,11230,(718) 258-1367,4.6,$$$,False,difarapizzany.com
7,Roberta's,"261 Moore St, Brooklyn, NY",Mushroom,Brooklyn,NY,11206,(718) 417-1118,4.5,$$,True,robertaspizza.com


In [23]:
query = "get pizza places in new york that likes pepperoni"

messages = [
    UserMessage(content=query, source="user"),
    AssistantMessage(content=find_similar_pizzas(query).to_string(), source="rag"),
]

response = await client.create(messages=messages)

print(response.content)

Here are some pizza places in New York that are known for their love of pepperoni:

1. **Joe's Pizza**
   - **Address:** 7 Carmine St, New York, NY 10014
   - **Phone Number:** (212) 366-1182
   - **Rating:** 4.5
   - **Price Range:** $
   - **Delivery:** Yes
   - **Website:** [joespizzanyc.com](http://joespizzanyc.com)

2. **Artichoke Basille's Pizza**
   - **Address:** 321 E 14th St, New York, NY 10003
   - **Phone Number:** (212) 228-2004
   - **Rating:** 4.3
   - **Price Range:** $$
   - **Delivery:** Yes
   - **Website:** [artichokepizza.com](http://artichokepizza.com)

3. **Prince Street Pizza**
   - **Address:** 27 Prince St A, New York, NY 10012
   - **Phone Number:** (212) 966-4100
   - **Rating:** 4.4
   - **Price Range:** $$
   - **Delivery:** Yes
   - **Website:** [princestreetpizzanyc.com](http://princestreetpizzanyc.com)

Enjoy your pepperoni pizza!
